In [1]:
import pandas as pd
import ast
import re
from pandas import json_normalize

In [2]:
df_events = pd.read_csv('./Data/liga brasil/pre_games_events.csv')
df_events.head()

,type,minute,extra,player,club,action,game_id
0,Goals,70,NaN,{'href': '/biel/profil/spieler/875328'},"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': '0:1', 'description': ', Right-foot...",4316187
1,Goals,72,NaN,{'href': '/wesley/profil/spieler/674732'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': '1:1', 'description': ', Left-foote...",4316187
2,Goals,83,NaN,{'href': '/fernando/profil/spieler/51174'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': '2:1', 'description': ', Header, 1....",4316187
3,Substitutions,46,NaN,{'href': '/lucca/profil/spieler/982549'},"{'name': 'Sport Club Internacional', 'href': '...","{'result': None, 'description': ', Tactical', ...",4316187
4,Substitutions,46,NaN,{'href': '/oscar-estupinan/profil/spieler/3882...,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': None, 'description': ', Tactical', ...",4316187


In [3]:
df_events.drop('extra',axis=1,inplace=True)

In [4]:
# Función para convertir texto a diccionario y extraer el número de 'href'
def extraer_id(texto):
    try:
        # Convertir texto a diccionario
        diccionario = ast.literal_eval(texto)
        # Obtener el valor de 'href'
        href = diccionario.get('href', '')
        # Usar una expresión regular para extraer el número
        match = re.search(r'\d+', href)
        return int(match.group(0)) if match else None
    except (ValueError, SyntaxError):
        return None

# Aplicar la función a la columna 'player'
df_events['player_id'] = df_events['player'].apply(extraer_id).astype(int)
df_events = df_events.drop('player',axis=1)

df_events.head()

,type,minute,club,action,game_id,player_id
0,Goals,70,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': '0:1', 'description': ', Right-foot...",4316187,875328
1,Goals,72,"{'name': 'Sport Club Internacional', 'href': '...","{'result': '1:1', 'description': ', Left-foote...",4316187,674732
2,Goals,83,"{'name': 'Sport Club Internacional', 'href': '...","{'result': '2:1', 'description': ', Header, 1....",4316187,51174
3,Substitutions,46,"{'name': 'Sport Club Internacional', 'href': '...","{'result': None, 'description': ', Tactical', ...",4316187,982549
4,Substitutions,46,"{'name': 'Esporte Clube Bahia', 'href': '/espo...","{'result': None, 'description': ', Tactical', ...",4316187,388266


In [5]:
# Definir una función para extraer el número de un texto
def extraer_numero(texto):
    if isinstance(texto, str):  # Verificar si el texto es una cadena de texto
        numero = re.search(r'/verein/(\d+)', texto)
        if numero:
            return numero.group(1)
    return None  # Devolver None si el texto no es una cadena de texto o no se encuentra un número

# Aplicar la función a la columna 'texto', omitiendo valores que no sean cadenas de texto
df_events['club_id'] = df_events['club'].apply(extraer_numero)
df_events.drop('club',axis=1,inplace=True)

In [6]:
df_events.head()

,type,minute,action,game_id,player_id,club_id
0,Goals,70,"{'result': '0:1', 'description': ', Right-foot...",4316187,875328,10010
1,Goals,72,"{'result': '1:1', 'description': ', Left-foote...",4316187,674732,6600
2,Goals,83,"{'result': '2:1', 'description': ', Header, 1....",4316187,51174,6600
3,Substitutions,46,"{'result': None, 'description': ', Tactical', ...",4316187,982549,6600
4,Substitutions,46,"{'result': None, 'description': ', Tactical', ...",4316187,388266,10010


In [7]:
# Aplica ast.literal_eval a cada elemento en la columna 'action'
df_events['action'] = df_events['action'].apply(lambda x: ast.literal_eval(x))

# Usar json_normalize para aplanar los diccionarios
df_normalized = json_normalize(df_events['action'])

# Concatenar el DataFrame original con el DataFrame normalizado
df_events = pd.concat([df_events, df_normalized], axis=1)

# Eliminar la columna original de diccionarios si ya no es necesaria
df_events.drop(columns=['action'], inplace=True)

df_events.head()

,type,minute,game_id,player_id,club_id,result,description,player_in.href,player_assist.href
0,Goals,70,4316187,875328,10010,0:1,", Right-footed shot, 1. Goal of the Season Ass...",None,/jean-lucas/leistungsdatendetails/spieler/5788...
1,Goals,72,4316187,674732,6600,1:1,", Left-footed shot, 1. Goal of the Season",None,None
2,Goals,83,4316187,51174,6600,2:1,", Header, 1. Goal of the Season Assist: , Corn...",None,/bruno-henrique/leistungsdatendetails/spieler/...
3,Substitutions,46,4316187,982549,6600,None,", Tactical",/wesley/profil/spieler/674732,None
4,Substitutions,46,4316187,388266,10010,None,", Tactical",/cauly/profil/spieler/162812,None


In [8]:
df_events.drop(columns=['result'], inplace=True)

In [9]:
# Definir una función para extraer el número de un texto
def extraer_numero(texto):
    if isinstance(texto, str):  # Verificar si el texto es una cadena de texto
        numero = re.search(r'\d+', texto)
        if numero:
            return numero.group()
    return None  # Devolver None si el texto no es una cadena de texto o no se encuentra un número

# Reemplaza valores nulos en 'player_in.href' y 'player_assist.href' con '0' antes de aplicar extraer_numero
df_events['player_in'] = df_events['player_in.href'].fillna('0').apply(extraer_numero).astype(int)
df_events['player_assist'] = df_events['player_assist.href'].fillna('0').apply(extraer_numero).astype(int)
df_events = df_events.drop('player_in.href',axis=1)
df_events = df_events.drop('player_assist.href',axis=1)

In [10]:
df_events.head()

,type,minute,game_id,player_id,club_id,description,player_in,player_assist
0,Goals,70,4316187,875328,10010,", Right-footed shot, 1. Goal of the Season Ass...",0,578872
1,Goals,72,4316187,674732,6600,", Left-footed shot, 1. Goal of the Season",0,0
2,Goals,83,4316187,51174,6600,", Header, 1. Goal of the Season Assist: , Corn...",0,277335
3,Substitutions,46,4316187,982549,6600,", Tactical",674732,0
4,Substitutions,46,4316187,388266,10010,", Tactical",162812,0


In [11]:
df_events.to_csv('./Data/liga brasil/game_events.csv',index=False)